# Welcome to Stable Diffusion WebUI 1.4! by [@altryne](https://twitter.com/altryne/) |  [![ko-fi](https://ko-fi.com/img/githubbutton_sm.svg)](https://ko-fi.com/N4N3DWMR1) 

This colab runs the latest webui version from the repo https://github.com/hlky/stable-diffusion-webui

---

If this colab helped you, support me on ko-fi and don't forget to subscribe to my awesome list https://github.com/altryne/awesome-ai-art-image-synthesis


## 1 -  Setup stage

In [1]:
!nvidia-smi -L

GPU 0: Quadro P5000 (UUID: GPU-12b1017c-5bab-e482-e14f-af55281cc1b2)


### 1.1 - Install dependencies

In [1]:
import sys

!{sys.executable} -m pip install -e .
!conda env update -n base -f environment.yaml 

Obtaining file:///notebooks
     |████████████████████████▋       | 597.1 MB 74.7 MB/s eta 0:00:0311 MB/s eta 0:00:06MB/s eta 0:00:04

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 776.4 MB 26 kB/s s eta 0:00:01████████████████████████████▍ | 737.2 MB 90.2 MB/s eta 0:00:01
     |████████████████████████████████| 17.1 MB 78.8 MB/s eta 0:00:01
  Running setup.py develop for latent-diffusion
Solving environment: done

libwebp-1.2.2        | 80 KB     | ##################################### | 100% 
conda-package-handli | 885 KB    | ##################################### | 100% 
git-2.34.1           | 6.0 MB    | ##################################### | 100% 
blas-1.0             | 6 KB      | ##################################### | 100% 
libssh2-1.10.0       | 274 KB    | ##################################### | 100% 
libwebp-base-1.2.2   | 440 KB    | ##################################### | 100% 
importlib_metadata-4 | 12 KB     | ##################################### | 100% 
jpeg-9e              | 240 KB    | ##################################### | 100% 
importlib-metadata-4 | 40 KB     | ##################################### |

### 1.2 Setup Upscalers - CFPGan and ESRGAN

**GFPGAN** Automatically correct distorted faces with a built-in GFPGAN option, fixes them in less than half a second

**ESRGAN** Boosts the resolution of images with a built-in RealESRGAN option

In [2]:
add_CFP = True
add_ESR = True

if add_CFP:
    %cd /notebooks/src/gfpgan/
    !{sys.executable} -m pip install basicsr facexlib yapf lmdb opencv-python pyyaml tb-nightly --no-deps
    !{sys.executable} setup.py develop
    !{sys.executable} -m pip install realesrgan
    !wget -nc https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models
if add_ESR:
    %cd /notebooks/src/realesrgan/
    !wget -nc https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
    !wget -nc https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth -P experiments/pretrained_models

%cd /notebooks/
!wget -nc https://github.com/matomo-org/travis-scripts/blob/master/fonts/Arial.ttf?raw=true -O arial.ttf

/notebooks/src/gfpgan
/root/miniconda/lib/python3.8/site-packages/setuptools/installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecated. Requirements should be satisfied by a PEP 517 installer.
  warnings.warn(
running develop
/root/miniconda/lib/python3.8/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/root/miniconda/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running egg_info
writing gfpgan.egg-info/PKG-INFO
writing dependency_links to gfpgan.egg-info/dependency_links.txt
writing requirements to gfpgan.egg-info/requires.txt
writing top-level names to gfpgan.egg-info/top_level.txt
reading manifest file 'gfpgan.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding

### 1.3 Load the stable-diffusion model

In [3]:
models_path = "/notebooks/models"
output_path = "/notebooks/output"

import os
os.makedirs(models_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)

print(f"models_path: {models_path}")
print(f"output_path: {output_path}")

models_path: /notebooks/models
output_path: /notebooks/output


In [4]:
!wget -nc https://files.mulliken.net/file/mulliken-public-files/stable-diffusion/sd-v1-4.ckpt -O {models_path}/sd-v1-4.ckpt

File ‘/notebooks/models/sd-v1-4.ckpt’ already there; not retrieving.


## 2 - Run the Stable Diffusion webui

### 2.1 Optional - Set webUI settings and configs before running 

In [5]:
share_password="password"

defaults="configs/webui/webui.yaml"

save_metadata = False
skip_grid = False
skip_save = False
optimized = True
optimized_turbo = True
no_verify_input = False
no_half = True
no_progressbar_hiding = True
extra_models_cpu = False 
# Run ESRGAN on the cpu?
esrgan_cpu = False
# Run GFPGAN on the cpu?
gfpgan_cpu = False


run_string_with_variables = {
 '--save-metadata': f'{save_metadata}',
 '--skip-grid': f'{skip_grid}',
 '--skip-save': f'{skip_save}',
 '--optimized': f'{optimized}',
 '--optimized-turbo': f'{optimized_turbo}',
 '--no-verify-input': f'{no_verify_input}',
 '--no-half': f'{no_half}',
 '--no-progressbar-hiding': f'{no_progressbar_hiding}',
 '--extra-models-cpu': f'{extra_models_cpu}',
 '--esrgan-cpu': f'{esrgan_cpu}',
 '--gfpgan-cpu': f'{gfpgan_cpu}'}

only_true_vars = {k for (k,v) in run_string_with_variables.items() if v == 'True'}
vars = " ".join(only_true_vars)


## 3 - Launch WebUI for stable diffusion

Be sure to open the *public link* in another tab to use the webui.

In [ ]:
%cd /notebooks
!{sys.executable} /notebooks/scripts/webui.py \
--ckpt '{models_path}/sd-v1-4.ckpt' \
--outdir '{output_path}' \
--share {vars} \
--precision full


/notebooks
Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth" to /root/miniconda/lib/python3.8/site-packages/facexlib/weights/detection_Resnet50_Final.pth

100%|█████████████████████████████████████████| 104M/104M [00:00<00:00, 115MB/s]
Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.2.2/parsing_parsenet.pth" to /root/miniconda/lib/python3.8/site-packages/facexlib/weights/parsing_parsenet.pth

100%|███████████████████████████████████████| 81.4M/81.4M [00:00<00:00, 128MB/s]
Loaded GFPGAN
Loaded RealESRGAN with model RealESRGAN_x4plus
Loading model from /notebooks/models/sd-v1-4.ckpt
Global Step: 470000
UNet: Running in eps-prediction mode
CondStage: Running in eps-prediction mode
Downloading: "https://github.com/DagnyT/hardnet/raw/master/pretrained/train_liberty_with_aug/checkpoint_liberty_with_aug.pth" to /root/.cache/torch/hub/checkpoints/checkpoint_liberty_with_aug.pth
100%|██████████████████████████████████